# GenVeg Main Script
GenVeg is a generalized vegetation growth model that simulates growth due to
photosynthesis as well as processes that control vegetation distribution such
as mortality, senescence, dispersal, and colonization. The model utilizes a 
mixture of existing and new landlab components to setup the model, simulates
relevant vegetation community dynamics with the local environment, and analyzes
results of the simulation. 

## Main model control set up
### Import components

In [12]:
import numpy as np, pandas as pd
import os, tempfile
from pathlib import Path
from datetime import date
from matplotlib.pyplot import title, show, figure, plot, subplot, xlabel, ylabel
from landlab import RasterModelGrid, imshow_grid, load_params
from landlab.data_record import DataRecord
from landlab.components.genveg import VegParams, PlantGrowth, GenVeg
from landlab.io.netcdf import write_netcdf

### Define time controls

In [13]:
sim_start_date = np.datetime64('2010-01-01')
tot_sim_len_y = 30
veg_time_step_d = 1
env_time_step_d = 1
dt=np.timedelta64(min([veg_time_step_d,env_time_step_d]),'D')
run_length_d=365

### Define filenames for input files
##### *Note should be located in same folder as this .ipynb

In [14]:
VegInputFile = 'GenVeg_Example_Simulation.xlsx'

### Read in parameter and data input files
Identify model processes to be included so all required parameters are defined and calculated

In [15]:
inp_file=Path(VegInputFile).absolute()
VegParams(fpath=inp_file,processes=['plantsize','dispersal','colonization','storage'])
#VegParams(processes=['plantsize','dispersal','colonization','storage','mortality'])
vegparams=load_params('veg_params.yml')
print(vegparams)


{'Corn': {'growparams': {'glucose_requirement': [1.444, 1.513, 1.463], 'growing_season_end': 243, 'growing_season_start': 121, 'k_light_extinct': 0.51, 'light_half_sat': 100, 'p_max': 0.0372, 'plant_part_min': [0.01, 0.1, 0.5], 'respiration_coefficient': [0.015, 0.015, 0.03], 'root_to_leaf_coeffs': [0.031, 0.951, 0], 'root_to_stem_coeffs': [-0.107, 1.098, 0.0216], 'senescence_start': 228}, 'plant_factors': {'angio_gymno': 'angiosperm', 'annual_perennial': 'annual', 'growth_form': 'graminoid', 'monocot_dicot': 'monocot', 'ptype': 'C4', 'species': 'Corn'}, 'sizeparams': {'max_height_stem': 1.0, 'max_n_stems': 3.0, 'max_plant_density': 34.0, 'total_cs_area_stems': 0.231}, 'storparams': {'wint_dieoff_roots': 0.25}}, 'Wheat': {'growparams': {'glucose_requirement': [1.444, 1.513, 1.463], 'growing_season_end': 305, 'growing_season_start': 244, 'k_light_extinct': 0.48, 'light_half_sat': 100, 'p_max': 0.5, 'plant_part_min': [0.01, 0.1, 0.5], 'respiration_coefficient': [0.015, 0.015, 0.03], 'roo

### Set up model grid using landlab RasterModelGrid functions

- Create the model grid
- Initialize live vegetation biomass and plant functional type
- Initialize the air temperature driver across the grid

##### **Note variable names should be consistent with landlab standards

In [16]:
#Create grid 10x10 node grid with 2 m node spacing and xy of origin in lat long roughly corresponding to Island Beach State Park
pg = RasterModelGrid((12, 12), 2.0, xy_of_reference=(-74.08,39.79))
pg.axis_units = ('m', 'm')

meantemp=[0.444444445,0.305555556,0.222222223,0.138888889,0.027777777,-0.083333334,-0.138888889,-0.25,-0.305555556,-0.416666667,-0.472222223,-0.527777778,-0.583333334,-0.638888889,-0.694444445,-0.75,-0.777777778,-0.805555556,-0.861111111,-0.861111111,-0.861111111,-0.888888889,-0.916666667,-0.916666667,-0.888888889,-0.861111111,-0.861111111,-0.833333333,-0.805555556,-0.777777778,-0.722222222,-0.666666667,-0.638888889,-0.583333333,-0.527777778,-0.472222222,-0.388888889,-0.333333334,-0.25,-0.166666667,-0.083333334,-0.027777777,0.083333334,0.166666667,0.277777778,0.361111111,0.472222222,0.583333333,0.666666667,0.777777778,0.888888889,1,1.083333334,1.194444445,1.305555556,1.416666667,1.527777778,1.611111111,1.694444445,1.805555556,1.916666667,2.027777778,2.138888889,2.25,2.333333334,2.444444445,2.555555556,2.638888889,2.75,2.861111111,2.944444445,3.055555556,3.166666667,3.25,3.361111112,3.444444445,3.555555556,3.694444445,3.777777778,3.888888889,4,4.083333334,4.222222223,4.333333334,4.444444445,4.583333334,4.694444445,4.833333333,4.944444444,5.083333334,5.222222223,5.333333334,5.5,5.638888889,5.777777778,5.916666667,6.083333334,6.222222222,6.38888889,6.555555554,6.722222222,6.88888889,7.055555554,7.222222222,7.38888889,7.555555554,7.722222222,7.88888889,8.083333334,8.250000002,8.444444443,8.638888891,8.805555555,8.972222223,9.166666666,9.333333335,9.527777778,9.694444446,9.888888887,10.08333334,10.25,10.44444444,10.61111111,10.77777778,10.97222222,11.13888889,11.27777778,11.47222222,11.63888889,11.80555556,11.97222222,12.16666667,12.33333333,12.47222222,12.63888889,12.80555555,13,13.16666667,13.33333333,13.47222222,13.63888889,13.80555556,13.97222222,14.13888889,14.33333334,14.47222222,14.63888889,14.80555556,14.97222222,15.16666667,15.33333334,15.5,15.66666667,15.86111112,16.02777778,16.22222222,16.38888889,16.58333334,16.75,16.91666667,17.13888889,17.30555556,17.47222223,17.66666667,17.86111112,18.02777778,18.22222223,18.38888889,18.58333334,18.75,18.91666667,19.08333334,19.27777778,19.41666667,19.58333334,19.75,19.91666667,20.08333334,20.19444445,20.36111112,20.47222223,20.63888889,20.75,20.86111112,20.97222223,21.08333334,21.19444445,21.30555556,21.38888889,21.47222223,21.55555556,21.61111111,21.69444445,21.75,21.80555556,21.83333334,21.88888889,21.94444445,21.94444445,22,22,22,22.02777778,22.02777778,22.02777778,22.02777778,22,22,21.97222222,21.97222222,21.91666667,21.91666667,21.86111111,21.86111111,21.80555556,21.75,21.72222222,21.66666667,21.61111111,21.58333334,21.52777778,21.47222223,21.41666667,21.36111112,21.30555556,21.25,21.16666667,21.11111111,21.05555556,20.97222222,20.91666667,20.83333333,20.75,20.69444445,20.58333334,20.5,20.44444445,20.33333334,20.25,20.13888889,20.05555556,19.94444444,19.80555556,19.69444445,19.58333334,19.44444445,19.33333334,19.19444445,19.08333334,18.91666667,18.80555556,18.63888889,18.5,18.33333333,18.19444445,18.02777778,17.86111111,17.69444445,17.52777778,17.36111111,17.19444445,16.97222223,16.80555556,16.61111112,16.44444445,16.25,16.08333334,15.88888889,15.66666667,15.47222222,15.30555556,15.11111112,14.91666667,14.72222222,14.55555556,14.36111111,14.13888889,13.94444444,13.77777778,13.58333333,13.41666667,13.22222222,13.05555555,12.88888889,12.72222222,12.52777778,12.36111111,12.19444444,12.02777778,11.88888889,11.72222222,11.55555556,11.38888889,11.25,11.08333333,10.91666667,10.77777778,10.63888889,10.5,10.33333334,10.19444444,10.05555556,9.916666666,9.750000002,9.611111112,9.472222221,9.361111111,9.194444442,9.055555557,8.916666666,8.777777776,8.611111112,8.472222222,8.333333331,8.194444446,8.027777777,7.888888887,7.750000001,7.583333333,7.444444442,7.277777779,7.138888889,6.97222222,6.805555555,6.638888891,6.500000001,6.333333332,6.166666669,6.027777778,5.861111111,5.694444445,5.527777778,5.361111111,5.194444445,5.027777778,4.861111111,4.694444445,4.527777778,4.361111111,4.194444445,4.027777778,3.861111111,3.722222222,3.555555556,3.388888889,3.222222222,3.083333334,2.916666667,2.75,2.611111112,2.444444445,2.305555556,2.138888889,2,1.833333333,1.722222222,1.555555556,1.416666667,1.305555556,1.166666667,1.027777778,0.888888889,0.777777778,0.666666667,0.555555556
]
dailyTotGlobRad=[22.3, 26.8, 32.2, 37.1, 40.0, 40.9, 40.4, 38.2, 33.9, 28.5, 23.3, 21.0]
                
#Initialize with a random mean air temperture 
_ = pg.add_field('air__temperature_C', meantemp[0]*np.ones(pg.number_of_cells), at='cell', units='C' )
_ = pg.add_field('radiation__net_flux', dailyTotGlobRad[0]*np.ones(pg.number_of_cells), at='cell', units='W/m^2')


Now we initialize plants. We can do this in multiple ways. Here we are going to seed two species in a grid with 1/2 occupied by Corn, 1/2 by wheat, at single and double planting densities, respectively

In [17]:
#Initialize plant functional type as burial tolerant stabilizer or dune builder and randomly place on grid
plant_species_init=['Corn','Wheat']
plant_list_1=['Corn','null','null','null']
plant_list_2=['null','null','Wheat','null']
plant_list_3=['Corn','Corn','null','null']
plant_list_4=['null','null','Wheat','Wheat']
plant_lists=[plant_list_1,plant_list_2,plant_list_3, plant_list_4]
plants_on_grid=[]
length_field_part=pg.number_of_cells/4
for plant_list in plant_lists:
    for i in range(int(length_field_part)):
        #if plants_on_grid.isempty
        plants_on_grid.append(plant_list)

_ = pg.add_field('vegetation__plant_species',plants_on_grid, at='cell', clobber=True)

#occupied_ids=np.where(pg.at_cell['vegetation__plant_species']=='BTS')

pidval=0
plants=pd.DataFrame(columns=['pid','species','cell_index'])
for cell in range(pg.number_of_cells):
    cell_index=cell
    cell_plants=pg['cell']['vegetation__plant_species'][cell]
    for plant in cell_plants:
        species=plant
        newrow=[pidval,species,cell_index]
        plants.loc[pidval]=newrow
        pidval += 1

pid=plants.pid.loc[plants['species']=='BTS'].to_numpy(int)
total_biomass=np.random.rand(pid.shape[0])
total_biomass2=np.random.rand(6)

species_set=np.vstack((pid,total_biomass))
species_set2=np.vstack((total_biomass2,total_biomass2))
species_set=np.transpose(species_set)
species_set2=np.transpose(species_set2)
biomass=np.concatenate((species_set,species_set2), axis=0)
print(biomass)
biomass=np.transpose(biomass)
print(biomass)

In [18]:
#This was a check to see if I could save a list as a field and it is possible so we can have multiple species per cell
#veg_biomass=np.where(pg.at_cell['vegetation__plant_species'] != 'null',1,0)*np.random.uniform(low=0, high=200,size=pg.number_of_cells)
#veg_biomass_part=veg_biomass/3
#veg_biomass_tot=[]
#for ind, item in enumerate(veg_biomass):
#    veg_biomass_tot.append([item, veg_biomass_part[ind],veg_biomass_part[ind],veg_biomass_part[ind]])
#print(veg_biomass_tot)

### Plot grid with plant distribution and initial biomass

Use RasterModelGrid plot functions to visualize initial model grid

#### Vegetation type 
- 0=Bare
- 1=Annual
- 2=Herbaceous Perennial 
- 3=Woody Perennial

In [19]:
#Need to pudate to use colormap based on unique strings
#imshow_grid(pg, 'vegetation__plant_species', at='cell',colorbar_label='Vegetation Type')
#show()

In [20]:
#imshow_grid(pg, 'vegetation__live_biomass', at='cell', colorbar_label='Live biomass g')
#show()

### Define model time variables
Determine basic model timestep and counter for model loop

sim_end = date(sim_start_date.year+tot_sim_len_y,sim_start_date.month,sim_start_date.day)
dt = min([veg_time_step_d,env_time_step_d])
#data['dt']=dt
day_freq = str(dt)+'D'
date_counter = pd.date_range(start=sim_start_date,end=sim_end,freq=day_freq).strftime('%Y-%m-%d').tolist()
step_counter=range(len(date_counter))
model_counter=tuple(zip(step_counter,date_counter))

Initialize crops PlantGrowth class with veg_params from Excel file

In [21]:
croprun=GenVeg(pg,dt,sim_start_date,vegparams)
print(croprun.plant_ID())



     pid  cell_index species  leaf_biomass  stem_biomass  root_biomass
0      0           0    Corn      0.538851      0.354252      0.484212
1      1           1    Corn      0.512786      0.334788      0.459614
2      2           2    Corn      0.576952      0.382971      0.520278
3      3           3    Corn      0.692121      0.471581      0.630014
4      4           4    Corn      0.568231      0.376370      0.512012
..   ...         ...     ...           ...           ...           ...
145  145          97   Wheat      0.609774      0.407956      0.551446
146  146          98   Wheat      0.612289      0.409880      0.553838
147  147          98   Wheat      0.621163      0.416677      0.562281
148  148          99   Wheat      0.602851      0.402667      0.544864
149  149          99   Wheat      0.684369      0.465536      0.622596

[150 rows x 6 columns]


## Main model loop script
This is the main model loop that controls the simulation that
- Advances model time 
- Updates environmental variables
- Simulates changes to vegetation communities
- Stores outputs

In [22]:
#This runs through fine for day 1 of growing season. We need to decide how to track current day
for i in range(run_length_d):
    #update temp attribute on grid
    pg.at_cell['air__temperature_C']=np.ones(pg.number_of_cells)*meantemp[i]
    #update solar radiation on grid
    croprun.run_one_step(dt)
    croprun.save_output()
    if (i/28).is_integer():
        display(croprun.plant_ID())    #if this is a save timestep, save grid object to netcdf
    #create record method in croprun that uses datarecord to save PlantGrowth object and grid at time
    #if i in savetime:
    #    croprun.record()

,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass
0,0,0,Corn,0.538851,0.354252,0.484212
1,1,1,Corn,0.512786,0.334788,0.459614
2,2,2,Corn,0.576952,0.382971,0.520278
3,3,3,Corn,0.692121,0.471581,0.630014
4,4,4,Corn,0.568231,0.376370,0.512012
...,...,...,...,...,...,...
145,145,97,Wheat,0.609774,0.407956,0.551446
146,146,98,Wheat,0.612289,0.409880,0.553838
147,147,98,Wheat,0.621163,0.416677,0.562281
148,148,99,Wheat,0.602851,0.402667,0.544864


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,0.538851,0.354252,0.484212,4
1,1,1,Corn,0.512786,0.334788,0.459614,4
2,2,2,Corn,0.576952,0.382971,0.520278,4
3,3,3,Corn,0.692121,0.471581,0.630014,4
4,4,4,Corn,0.568231,0.376370,0.512012,4
...,...,...,...,...,...,...,...
145,145,97,Wheat,0.609774,0.407956,0.551446,4
146,146,98,Wheat,0.612289,0.409880,0.553838,4
147,147,98,Wheat,0.621163,0.416677,0.562281,4
148,148,99,Wheat,0.602851,0.402667,0.544864,4


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,0.538851,0.354252,0.484212,8
1,1,1,Corn,0.512786,0.334788,0.459614,8
2,2,2,Corn,0.576952,0.382971,0.520278,8
3,3,3,Corn,0.692121,0.471581,0.630014,8
4,4,4,Corn,0.568231,0.376370,0.512012,8
...,...,...,...,...,...,...,...
145,145,97,Wheat,0.609774,0.407956,0.551446,8
146,146,98,Wheat,0.612289,0.409880,0.553838,8
147,147,98,Wheat,0.621163,0.416677,0.562281,8
148,148,99,Wheat,0.602851,0.402667,0.544864,8


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,0.538851,0.354252,0.484212,12
1,1,1,Corn,0.512786,0.334788,0.459614,12
2,2,2,Corn,0.576952,0.382971,0.520278,12
3,3,3,Corn,0.692121,0.471581,0.630014,12
4,4,4,Corn,0.568231,0.376370,0.512012,12
...,...,...,...,...,...,...,...
145,145,97,Wheat,0.609774,0.407956,0.551446,12
146,146,98,Wheat,0.612289,0.409880,0.553838,12
147,147,98,Wheat,0.621163,0.416677,0.562281,12
148,148,99,Wheat,0.602851,0.402667,0.544864,12


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,0.538851,0.354252,0.484212,16
1,1,1,Corn,0.512786,0.334788,0.459614,16
2,2,2,Corn,0.576952,0.382971,0.520278,16
3,3,3,Corn,0.692121,0.471581,0.630014,16
4,4,4,Corn,0.568231,0.376370,0.512012,16
...,...,...,...,...,...,...,...
145,145,97,Wheat,0.609774,0.407956,0.551446,16
146,146,98,Wheat,0.612289,0.409880,0.553838,16
147,147,98,Wheat,0.621163,0.416677,0.562281,16
148,148,99,Wheat,0.602851,0.402667,0.544864,16


...

,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,5.903529,5.749383,5.997482,36
1,1,1,Corn,5.892429,5.736420,5.985585,36
2,2,2,Corn,5.918509,5.766897,6.013545,36
3,3,3,Corn,5.956734,5.811700,6.054566,36
4,4,4,Corn,5.915201,5.763027,6.009997,36
...,...,...,...,...,...,...,...
145,145,97,Wheat,7.460249,7.400191,7.615035,36
146,146,98,Wheat,7.464593,7.405542,7.619766,36
147,147,98,Wheat,7.479739,7.424210,7.636259,36
148,148,99,Wheat,7.448167,7.385315,7.601883,36


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,5.903529,5.749383,5.997482,40
1,1,1,Corn,5.892429,5.736420,5.985585,40
2,2,2,Corn,5.918509,5.766897,6.013545,40
3,3,3,Corn,5.956734,5.811700,6.054566,40
4,4,4,Corn,5.915201,5.763027,6.009997,40
...,...,...,...,...,...,...,...
145,145,97,Wheat,11.536262,12.608284,12.073743,40
146,146,98,Wheat,11.536613,12.608860,12.074172,40
147,147,98,Wheat,11.537839,12.610880,12.075676,40
148,148,99,Wheat,11.535288,12.606686,12.072550,40


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,5.903529,5.749383,5.997482,44
1,1,1,Corn,5.892429,5.736420,5.985585,44
2,2,2,Corn,5.918509,5.766897,6.013545,44
3,3,3,Corn,5.956734,5.811700,6.054566,44
4,4,4,Corn,5.915201,5.763027,6.009997,44
...,...,...,...,...,...,...,...
145,145,97,Wheat,12.106462,13.375838,12.705252,44
146,146,98,Wheat,12.106593,13.376122,12.705439,44
147,147,98,Wheat,12.107049,13.377119,12.706092,44
148,148,99,Wheat,12.106099,13.375052,12.704735,44


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,5.903529,5.749383,5.997482,48
1,1,1,Corn,5.892429,5.736420,5.985585,48
2,2,2,Corn,5.918509,5.766897,6.013545,48
3,3,3,Corn,5.956734,5.811700,6.054566,48
4,4,4,Corn,5.915201,5.763027,6.009997,48
...,...,...,...,...,...,...,...
145,145,97,Wheat,12.106462,13.375838,12.705252,48
146,146,98,Wheat,12.106593,13.376122,12.705439,48
147,147,98,Wheat,12.107049,13.377119,12.706092,48
148,148,99,Wheat,12.106099,13.375052,12.704735,48


,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,5.903529,5.749383,5.997482,52
1,1,1,Corn,5.892429,5.736420,5.985585,52
2,2,2,Corn,5.918509,5.766897,6.013545,52
3,3,3,Corn,5.956734,5.811700,6.054566,52
4,4,4,Corn,5.915201,5.763027,6.009997,52
...,...,...,...,...,...,...,...
145,145,97,Wheat,12.106462,13.375838,12.705252,52
146,146,98,Wheat,12.106593,13.376122,12.705439,52
147,147,98,Wheat,12.107049,13.377119,12.706092,52
148,148,99,Wheat,12.106099,13.375052,12.704735,52


In [23]:
display(croprun.plant_df())

,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass,timestep
0,0,0,Corn,5.903529,5.749383,5.997482,52
1,1,1,Corn,5.892429,5.736420,5.985585,52
2,2,2,Corn,5.918509,5.766897,6.013545,52
3,3,3,Corn,5.956734,5.811700,6.054566,52
4,4,4,Corn,5.915201,5.763027,6.009997,52
...,...,...,...,...,...,...,...
145,145,97,Wheat,12.106462,13.375838,12.705252,52
146,146,98,Wheat,12.106593,13.376122,12.705439,52
147,147,98,Wheat,12.107049,13.377119,12.706092,52
148,148,99,Wheat,12.106099,13.375052,12.704735,52


In [24]:
temp_dir = tempfile.mkdtemp()
os.chdir(temp_dir)
write_netcdf("test.nc", pg, format="NETCDF3_64BIT", time=step_counter[0])

NameError: name 'step_counter' is not defined